In [1]:
from nltk.corpus import stopwords
#talvez seja necessário baixar as stopwords com o comando no CMD: python -m nltk.downloader stopwords
from string import punctuation
from nltk.tokenize import TweetTokenizer
import re

#______________________________________________________________________________________________________________________#

#Instância um objeto do tipo TweetTokenizer que já deixa tudo em minúsculo, trata espaços,  
# e elimina menções (mas não tira números).
tweet_tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True, preserve_case=False) 

#definição das stopwords (inclui pontuação para eliminá-la, caso apareça)
stopwords = set(stopwords.words('portuguese') + list(punctuation))

def tokenizar_texto(texto_completo):
    texto_tokenizado_lst = tweet_tokenizer.tokenize(texto_completo)
    palavras_sem_stopwords = [palavra for palavra in texto_tokenizado_lst if palavra not in stopwords]
    return palavras_sem_stopwords

In [2]:
import nltk
import pandas as pd
from collections import defaultdict
from IPython.display import clear_output

#______________________________________________________________________________________________________________________________#

def processar_tweets(arquivo,size,colunas,chunk_size = 10):#retorna um dataframe com size tweets
    # parametros: 
    # arquivo = str, path/nome do arquivo para leitura dos tweets
    # size = int, numero de tweets a serem lidos
    # columns = list, lista de strings com nome das colunas a serem lidas
    # chunk_size = int,tamanho dos chunks
    contador = 0
    num = size/chunk_size
    dados = pd.read_json(arquivo, convert_dates=False, lines=True, chunksize=chunk_size)
    tweets = pd.DataFrame(columns=colunas)
    for df_tweets_retweets in dados: ##->> Acessa os dados dos tweets e retweets de um chunk, gerando um dataframe do tamanho do chunk.                         
        contador += 1
        if contador<=num:
            tweets=tweets.append(df_tweets_retweets[colunas])
        else:
            return tweets
def bigrama_em_data(bigrama,df_tweets):#retorna uma serie com dia associado a num de ocorrencias
    df_tweets['date'] = [d.get('$date').split('T')[0] for d in df_tweets['created_at']]
    df_tweets.set_index('date',inplace=True)
    ocorrencias = pd.Series(name='ocorrencias bigrama')
    for data in df_tweets.index: #Percorre esse dataframe agrupado por data selecionando apenas o texto de cada tweet .
        ocorrencias[data] = 0
        for text_tweet in df_tweets['text'][data]:
            bigramas_tweet = list(nltk.bigrams(tokenizar_texto(text_tweet))) 
            if bigrama in bigramas_tweet: #só os tweets que contenham o bigrama ('distanciamento', 'social')
                ocorrencias[data] += bigramas_tweet.count(bigrama)
    return ocorrencias
print(processar_tweets('tweets_222_23abr2020.json',100,['text','created_at']))
bigrama_em_data(('distanciamento', 'social'),processar_tweets('tweets_222_23abr2020.json',100,['text','created_at']))

                                                 text  \
0   @gerlainesouza @andeersonreis Efeitos da quare...   
1   RT @viiih2906: " A quarentena acabou, Já pode ...   
2   RT @rubbgc: 2020 tinha tudo p ser meu ano 😖 co...   
3   O representante da província de Xangai - MARCO...   
4   RT @andreawerner_: “A fome vai matar muito mai...   
..                                                ...   
95  RT @desiretodie: não consigo entender pq vc es...   
96        RT @pqplaurina: quando a quarentena acabar:   
97  RT @EnockFormiga: Letalidade do Covid-19\nTaxa...   
98  @Tropa_Furacao Dos msm criadores de "acabei de...   
99  @allantercalivre @realDonaldTrump Leandro, pub...   

                                   created_at  
0   {'$date': '2020-04-11T20:40:10.000-0300'}  
1   {'$date': '2020-04-11T20:40:10.000-0300'}  
2   {'$date': '2020-04-11T20:40:10.000-0300'}  
3   {'$date': '2020-04-11T20:40:10.000-0300'}  
4   {'$date': '2020-04-11T20:40:10.000-0300'}  
..                         

C:\Users\vicda\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


2020-04-11    1
Name: ocorrencias bigrama, dtype: int64